# Modeling the Family Security Act
* Repeal SALT, CDCC, TANF


In [3]:
from openfisca_us import Microsimulation, IndividualSim


In [82]:
from openfisca_us.model_api import *
from openfisca_us.variables.household.demographic.tax_unit.filing_status import (
    FilingStatus,
)
from openfisca_us.tools.baseline_variables import baseline_variables

# Is cdcc the CDCTC?
REPLACED_PROGRAMS = ["cdcc", "ctc", "salt_deduction", "tanf"]

YOUNG_CHILD_AMOUNT = 350
CHILD_AMOUNT = 350
CAP = 1_250
PHASE_OUT_START_SINGLE = 200_000
PHASE_OUT_START_MARRIED = 400_000
PHASE_OUT_RATE = 0.05


class fsa(Variable):
    value_type = float
    entity = TaxUnit
    definition_period = YEAR
    label = "Family Security Act"

    def formula(tax_unit, period, parameters):
        # $350/mo for children under 6, $250/mo for children 6-17.
        person = tax_unit.members
        age = person("age", period)
        children_under_6 = age < 6
        children_6_17 = (age >= 6) & (age < 18)
        pre_cap = (children_under_6 * YOUNG_CHILD_AMOUNT) + (
            children_6_17 * CHILD_AMOUNT
        )
        capped = min_(tax_unit.sum(pre_cap), CAP)
        # Phase out at 5% above $200k for single filers and $400k for married filers.
        agi = tax_unit("adjusted_gross_income", period)
        filing_status = tax_unit("filing_status", period)
        phase_out_start = where(filing_status == FilingStatus.SINGLE, PHASE_OUT_START_SINGLE, PHASE_OUT_START_MARRIED)
        excess = max_(agi - phase_out_start, 0)
        # TODO: Phase out at $50 per $1,000 instead of linearly.
        phased_out = min_(excess * PHASE_OUT_RATE, capped)
        return (capped - phased_out) * 12


# Override filing status to move HOH to SINGLE.
class filing_status(Variable):
    def formula(tax_unit, period, parameters):
        has_spouse = add(tax_unit, period, ["is_tax_unit_spouse"]) > 0
        return where(has_spouse, FilingStatus.JOINT, FilingStatus.SINGLE)


class income_tax(Variable):
    def formula(tax_unit, period, parameters):
        original = baseline_variables["income_tax"].formula(tax_unit, period)
        return original - tax_unit("fsa", period)


# TODO: Separate into multiple reforms.
class add_fsa(Reform):
    def apply(self):
        for program in REPLACED_PROGRAMS:
            self.neutralize_variable(program)
        self.add_variable(fsa)
        self.update_variable(filing_status)
        self.update_variable(income_tax)

In [90]:
# ms = Microsimulation(year=2020)
isb = IndividualSim(year=2022)
isr = IndividualSim(add_fsa, year=2022)

for individual_sim in [isb, isr]:
    individual_sim.add_person(name="parent1", age=25)
    # individual_sim.add_person(name="parent2", age=25)
    individual_sim.add_person(name="child1", age=5)
    individual_sim.add_person(name="child2", age=5)
    members = ["parent1", "parent2", "child1", "child2"]
    members = ["parent1", "child1", "child2"]
    individual_sim.add_tax_unit(
        name="tax_unit",
        members=members,
        premium_tax_credit=0,
    )
    individual_sim.add_spm_unit(
        name="spm_unit",
        members=members,
        childcare_expenses=6_000,
        is_tanf_enrolled=True,
    )
    individual_sim.add_household(
        name="household",
        members=members,
        state_code="MA",
    )
    individual_sim.vary("employment_income", max=1_000_000)


In [91]:
import pandas as pd

df = pd.DataFrame(
    dict(
        employment_income=isb.calc("employment_income")[0],
        net_income_baseline=isb.calc("spm_unit_net_income")[0] + isb.calc("tanf")[0],
        net_income_fsa=isr.calc("spm_unit_net_income")[0],
    )
)

df["diff"] = df["net_income_fsa"] - df["net_income_baseline"]


In [92]:
import plotly.express as px

px.line(df, x="employment_income", y=["net_income_baseline", "net_income_fsa"])

In [93]:
px.line(df, x="employment_income", y="diff")

In [67]:
px.line(df, x="employment_income", y=isb.calc("tanf")[0])